In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Carica il dataset Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)

# Forza uno squilibrio rimuovendo alcune righe della classe 2 (Virginica)
X_class_0 = X[y == 0]
y_class_0 = y[y == 0]
X_class_1 = X[y == 1]
y_class_1 = y[y == 1]
X_class_2 = X[y == 2][:10]
y_class_2 = y[y == 2][:10]

X_imbalanced = pd.concat([X_class_0, X_class_1, X_class_2])
y_imbalanced = pd.concat([y_class_0, y_class_1, y_class_2])

# Dividi il dataset in training e test set
X_train, X_test, y_train, y_test = train_test_split(X_imbalanced, y_imbalanced, test_size=0.2, random_state=42, stratify=y_imbalanced)

# Applica l'oversampling alla classe minoritaria nel training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crea e addestra il modello con albero di decisione
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_res, y_train_res)

# Effettua predizioni sui dati originali (test set)
y_pred = clf.predict(X_test)

# Stampa i risultati
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9090909090909091
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.50      0.50      0.50         2

    accuracy                           0.91        22
   macro avg       0.80      0.80      0.80        22
weighted avg       0.91      0.91      0.91        22



In [5]:
y_test.value_counts()

0    10
1    10
2     2
Name: count, dtype: int64